# Build FAISS Index for Image Search

This notebook builds a FAISS index from images located in `data_dir/index_images` folder.

The process:
1. Load configuration from `configs/configs.yaml`
2. Extract features from all images using pre-trained CNN model
3. Build FAISS index for fast similarity search
4. Save embeddings and index to cache

In [1]:
import yaml
import sys
from pathlib import Path

# Add parent directory to path to import modules
sys.path.insert(0, str(Path.cwd().parent))

from settings import settings
from ml import ImageSearchEngine

In [2]:
parent_dir = Path.cwd().parent
config_path = parent_dir / "configs" / "configs.yaml"
data_dir = parent_dir / "data"

# Configuration is now loaded from settings.py
print("Current configuration:")
print(f"  Device: {settings.ml.device}")
print(f"  Model: {settings.ml.model_name}")
print(f"  Image size: {settings.ml.img_size}")
print(f"  Batch size: {settings.ml.batch_size}")
print(f"  Embedding dim: {settings.ml.embedding_dim}")
print(f"  Similarity metric: {settings.ml.similarity_metric}")
print(f"  Data directory: {settings.ml.data_dir}")
print(f"  Cache directory: {settings.ml.cache_dir}")

In [3]:
# Set data directory to index_images subfolder
base_data_dir = Path(settings.ml.data_dir)
index_images_dir = base_data_dir / "images"

print(f"Base data directory: {base_data_dir}")
print(f"Index images directory: {index_images_dir}")
print(f"Directory exists: {index_images_dir.exists()}")

Base data directory: /home/nedogeek/Documents/code/smartdiet/data/data
Index images directory: /home/nedogeek/Documents/code/smartdiet/data/data/images
Directory exists: True


In [4]:
# Note: Settings are now loaded from configs/configs.yaml via settings.py
# No need to create Config object manually
print("\nUsing settings from configs/configs.yaml")
print(f"All configuration is managed centrally via settings.py")


Using settings from configs/configs.yaml
All configuration is managed centrally via settings.py


In [6]:
from pydantic import BaseModel

class Config(BaseModel):
    device: str
    img_size: int
    batch_size: int
    embedding_dim: int
    model_name: str
    similarity_metric: str
    data_dir: str
    cache_dir: str
    log_level: str

In [ ]:
# Create Config object
config = Config(
    device=settings.ml.device,
    img_size=settings.ml.img_size,
    batch_size=settings.ml.batch_size,
    embedding_dim=settings.ml.embedding_dim,
    model_name=settings.ml.model_name,
    similarity_metric=settings.ml.similarity_metric,
    data_dir=str(index_images_dir),  # Use index_images subfolder
    cache_dir=settings.ml.cache_dir,
    log_level=settings.logging.level
)

print(f"\nConfiguration created:")
print(f"  Device: {config.device}")
print(f"  Model: {config.model_name}")
print(f"  Image size: {config.img_size}")
print(f"  Batch size: {config.batch_size}")
print(f"  Embedding dim: {config.embedding_dim}")
print(f"  Similarity metric: {config.similarity_metric}")
print(f"  Data directory: {config.data_dir}")
print(f"  Cache directory: {config.cache_dir}")


Configuration created:
  Device: cpu
  Model: resnet50
  Image size: 224
  Batch size: 32
  Embedding dim: 2048
  Similarity metric: cosine
  Data directory: /home/nedogeek/Documents/code/smartdiet/data/data/images
  Cache directory: /app/cache


## 2. Check Dataset

In [11]:
# Verify dataset exists and count images
if not index_images_dir.exists():
    print(f"⚠️ WARNING: Directory {index_images_dir} does not exist!")
    print(f"Please create it and add product images in subdirectories.")
else:
    # Count images and products
    image_extensions = {'.jpg', '.jpeg', '.png', '.webp'}
    products = [d for d in index_images_dir.iterdir() if d.is_dir()]
    
    total_images = 0
    for product_dir in products:
        images = [f for f in product_dir.rglob("*") if f.suffix.lower() in image_extensions]
        total_images += len(images)
        print(f"  {product_dir.name}: {len(images)} images")
    
    print(f"\n📊 Total: {len(products)} products, {total_images} images")

  69972: 5 images
  69332: 5 images
  9041: 5 images
  43181: 5 images
  56510: 5 images
  33537: 5 images
  9777: 5 images
  32635: 5 images
  32066: 5 images
  70631: 5 images
  14674: 5 images
  39081: 5 images
  52226: 5 images
  35934: 5 images
  48705: 5 images
  20415: 5 images
  50700: 5 images
  67909: 5 images
  58773: 5 images
  42056: 5 images
  9690: 5 images
  9774: 5 images
  35141: 5 images
  43393: 5 images
  70656: 5 images
  61961: 5 images
  41630: 5 images
  50536: 5 images
  7963: 5 images
  53199: 5 images
  39082: 5 images
  43765: 5 images
  26747: 5 images
  43308: 5 images
  10687: 5 images
  4091: 5 images
  26553: 5 images
  27508: 5 images
  9463: 5 images
  33563: 5 images
  6351: 5 images
  74979: 5 images
  7729: 5 images
  75017: 5 images
  26599: 5 images
  51879: 5 images
  56468: 5 images
  53974: 5 images
  7642: 5 images
  34901: 5 images
  42273: 5 images
  7677: 5 images
  15216: 5 images
  6610: 5 images
  56716: 5 images
  69909: 5 images
  76

# Initialize the search engine (no config parameter needed)
print("Initializing Image Search Engine...")
search_engine = ImageSearchEngine()
print("✅ Search engine initialized")

In [13]:
# Initialize the search engine
print("Initializing Image Search Engine...")
search_engine = ImageSearchEngine()
print("✅ Search engine initialized")

2025-12-07 17:25:50 [INFO] ml.image_search_engine: Loading resnet50 model...


Initializing Image Search Engine...
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/nedogeek/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:02<00:00, 42.7MB/s]
2025-12-07 17:25:53 [INFO] ml.image_search_engine: Model loaded successfully on cpu


✅ Search engine initialized


## 4. Build FAISS Index

This step will:
- Load all images from the dataset
- Extract features using the pre-trained model
- Create FAISS index
- Save embeddings to cache

**Note**: This may take several minutes depending on dataset size.

In [14]:
# Build index (force rebuild to ignore cache)
print("Building FAISS index...")
print("This may take a few minutes depending on dataset size...\n")

search_engine.build_index(force_rebuild=True)

print("\n✅ Index built successfully!")

2025-12-07 17:25:56 [INFO] ml.image_search_engine: Building index from scratch...
2025-12-07 17:25:56 [INFO] ml.image_search_engine: Loaded 20414 images from 1 categories


Building FAISS index...
This may take a few minutes depending on dataset size...



Extracting features: 100%|██████████| 638/638 [28:37<00:00,  2.69s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/app/cache/embeddings.pkl'

## 5. Verify Index and Get Statistics

In [15]:
# Get dataset statistics
stats = search_engine.get_statistics()

print("📊 Dataset Statistics:")
print(f"  Total images: {stats.get('total_images', 0)}")
print(f"  Unique products: {stats.get('unique_products', 0)}")
print(f"  Avg images per product: {stats.get('avg_images_per_product', 0):.2f}")

print(f"\n🔝 Top 10 products by image count:")
for product_id, count in stats.get('product_distribution', {}).items():
    print(f"  {product_id}: {count} images")

📊 Dataset Statistics:
  Total images: 20414
  Unique products: 1
  Avg images per product: 20414.00

🔝 Top 10 products by image count:
  images: 20414 images


In [16]:
# Verify FAISS index
if search_engine.index is not None:
    print(f"\n✅ FAISS index created successfully")
    print(f"  Index type: {type(search_engine.index).__name__}")
    print(f"  Total vectors: {search_engine.index.ntotal}")
    print(f"  Vector dimension: {search_engine.index.d}")
else:
    print("\n⚠️ WARNING: Index not created")


⚠️ WARNING: Index not created


## 6. Test Search (Optional)

Test the search functionality with a sample image from the dataset.

In [ ]:
# Test search with first image from dataset
if search_engine.image_paths:
    test_image_path = search_engine.image_paths[0]
    print(f"Testing search with image: {test_image_path}")
    
    results = search_engine.search(test_image_path, top_k=5)
    
    print(f"\n🔍 Top {len(results)} similar products:")
    for result in results:
        print(f"  Rank {result['rank']}: {result['product_id']}")
        print(f"    Similarity: {result['similarity']:.4f}")
        print(f"    Distance: {result['distance']:.4f}")
        print(f"    Image: {result['image_path']}")
        print()
else:
    print("No images in dataset to test")

# Check cache file
cache_file = Path(settings.ml.cache_dir) / "embeddings.pkl"

if cache_file.exists():
    cache_size_mb = cache_file.stat().st_size / (1024 * 1024)
    print(f"✅ Cache file saved: {cache_file}")
    print(f"  Size: {cache_size_mb:.2f} MB")
    print(f"\n💡 The API will load from this cache on startup.")
    print(f"   To rebuild, delete the cache file or use force_rebuild=True")
else:
    print(f"⚠️ Cache file not found: {cache_file}")

# Save index and metadata to specified files
import faiss
import pickle

index_file = Path("smart_diet_v0.1.index")
metadata_file = Path("metadata.pkl")

# Save FAISS index
if search_engine.index is not None:
    faiss.write_index(search_engine.index, str(index_file))
    print(f"\n✅ FAISS index saved to: {index_file}")
    index_size_mb = index_file.stat().st_size / (1024 * 1024)
    print(f"  Size: {index_size_mb:.2f} MB")
else:
    print("\n⚠️ No index to save")

# Save metadata (image paths and product IDs)
metadata = {
    'image_paths': search_engine.image_paths,
    'product_ids': search_engine.product_ids,
    'config': {
        'model_name': settings.ml.model_name,
        'embedding_dim': settings.ml.embedding_dim,
        'similarity_metric': settings.ml.similarity_metric,
        'img_size': settings.ml.img_size
    }
}

with open(metadata_file, 'wb') as f:
    pickle.dump(metadata, f)

print(f"\n✅ Metadata saved to: {metadata_file}")
metadata_size_mb = metadata_file.stat().st_size / (1024 * 1024)
print(f"  Size: {metadata_size_mb:.2f} MB")
print(f"  Contains: {len(metadata['image_paths'])} image paths, {len(set(metadata['product_ids']))} unique products")

In [ ]:
# Check cache file
cache_file = Path(config.cache_dir) / "embeddings.pkl"

if cache_file.exists():
    cache_size_mb = cache_file.stat().st_size / (1024 * 1024)
    print(f"✅ Cache file saved: {cache_file}")
    print(f"  Size: {cache_size_mb:.2f} MB")
    print(f"\n💡 The API will load from this cache on startup.")
    print(f"   To rebuild, delete the cache file or use force_rebuild=True")
else:
    print(f"⚠️ Cache file not found: {cache_file}")

In [ ]:
# Save index and metadata to specified files
import faiss
import pickle

index_file = data_dir / "smart_diet_v0.1.index"
metadata_file = data_dir / "metadata.pkl"

# Save FAISS index
if search_engine.index is not None:
    faiss.write_index(search_engine.index, str(index_file))
    print(f"✅ FAISS index saved to: {index_file}")
    index_size_mb = index_file.stat().st_size / (1024 * 1024)
    print(f"  Size: {index_size_mb:.2f} MB")
else:
    print("⚠️ No index to save")

# Save metadata (image paths and product IDs)
metadata = {
    'image_paths': search_engine.image_paths,
    'product_ids': search_engine.product_ids,
    'config': {
        'model_name': config.model_name,
        'embedding_dim': config.embedding_dim,
        'similarity_metric': config.similarity_metric,
        'img_size': config.img_size
    }
}

with open(metadata_file, 'wb') as f:
    pickle.dump(metadata, f)

print(f"\n✅ Metadata saved to: {metadata_file}")
metadata_size_mb = metadata_file.stat().st_size / (1024 * 1024)
print(f"  Size: {metadata_size_mb:.2f} MB")
print(f"  Contains: {len(metadata['image_paths'])} image paths, {len(set(metadata['product_ids']))} unique products")

## ✅ Done!

The FAISS index has been built and cached. You can now start the API server:

```bash
python app.py
```

The API will automatically load the cached embeddings on startup.